# 第四部分：Milvus 进阶实战

欢迎来到 Milvus Workshop 的第四部分！

这一部分主要是关于 Milvus 可观测性、运维、调优等实战，通过这部分能帮助我们更好得了解 Milvus 运行状态，保障 Milvus 稳定运行，且能更好得进行性能调优。

## 4.1 Milvus 可观测性运维实战

> 本节参考官方文档：[Milvus 监控、警报和日志](https://milvus.io/docs/zh/monitor_overview.md)

### 本节内容概览

- **1. 可观测性架构理解**：深入理解 Milvus 可观测性架构设计和核心组件作用
- **2. 关键指标分析**：掌握 Milvus 指标命名规范和标签体系，学会分析关键性能指标
- **3. 组件部署实践**：在集群中部署和配置完整的可观测性组件栈
- **4. 监控告警配置**：配置监控面板和告警规则，掌握故障排查方法

### 环境要求
- ✅ 确保已安装并运行 Milvus 集群
- ✅ 确保 K8s 集群内有足够的资源可安装 Prometheus + Loki + Jaeger + Grafana 以及对应的收集器

### 4.1.1 Milvus 可观测性架构和核心组件

可观测性（Observability）是现代分布式系统运维的核心概念，它通过三大支柱来帮助我们理解系统运行状态：

**三大支柱**：
- 📊 **指标（Metrics）**：数值化的系统状态数据，如 QPS、延迟、错误率等
- 📝 **日志（Logs）**：结构化的文本记录，记录系统运行过程中的关键事件
- 🔍 **追踪（Tracing）**：请求级别的调用链路追踪，帮助定位性能瓶颈

Milvus 可观测性架构采用分层设计，从下到上分为四层：

**Kubernetes 集群**
- `集群节点`
- `Milvus 集群`：Milvus 各个组件

 **数据采集层**：
- `Exporters`：采集节点和 Milvus 各组件暴露出来的各种指标
- `Promtail`：采集 Kubernetes 集群中的日志数据
- `Jaeger Agent`：采集分布式追踪数据，收集服务间的调用链路信息
 
**数据存储层**：
- `Prometheus`：存储时序指标数据
- `Loki`：存储日志数据
- `Jaeger`：存储分布式追踪数据
- `AlertManager`：告警管理，支持各种告警聚合、分组和路由等
 
**可视化层**：
- `Grafana`：统一的可视化监控面板，展示指标、日志和追踪数据
- `IM / Email`：告警通知渠道（邮件、钉钉、飞书等）


**可观测性架构图**

![Milvus可观测性架构图](./images/milvus-observability.png)


### 4.1.2. Milvus Metrics 说明

**Metrics 命名结构**

Milvus 中的有效 Metrics 名包含三个要素，用 `_` 连接：

```
namespace_subsystem_name
```

- **namespace**: Milvus 组件所在的命名空间（以下假设为 `Milvus`）
- **subsystem**: Metrics 所属的组件角色
- **name**: 具体的 Metrics 名称

**系统组件 (Subsystem)**

根据 Metrics 所属的角色，subsystem 包含以下八种：

| 组件 | 作用 |
|------|------|
| `proxy`      | 代理节点 - 请求入口和负载均衡 |
| `rootcoord`  | 根协调器 - 集群拓扑管理      |
| `querycoord` | 查询协调器 - 查询节点管理    |
| `querynode`  | 查询节点 - 执行搜索和查询    |
| `datacoord`  | 数据协调器 - 数据写入管理    |
| `datanode`   | 数据节点 - 数据写入和持久化  |
| `indexcoord` | 索引协调器 - 索引构建管理    |
| `indexnode`  | 索引节点 - 执行索引构建      |

**Metrics 命名示例**

```bash
# 代理节点搜索向量数量
milvus_proxy_search_vectors_count

# 查询节点的搜索请求延迟
milvus_querynode_sq_req_latency

# 查询协调器累计加载请求数
milvus_querycoord_load_req_count
```

更多更完整的 Metrics 可以参考官网：[Milvus Metrics 控制面板](https://milvus.io/docs/zh/metrics_dashboard.md)

**Metrics 类型**

Prometheus 支持四种 Metrics 类型：

| 类型 | 特点 | 用途 | 示例 |
|------|------|------|------|
| **Counter** | 累积类型，只能增加或重置为0 | 计数器，如累计查询的向量数 | `milvus_proxy_search_vectors_count` |
| **Gauge** | 可增可减的瞬时值 | 当前状态，如调度程序估算的每个查询节点的 CPU 占用率 | `milvus_querynode_estimate_cpu_usage` |
| **Histogram** | 基于可配置桶的分布统计 | 搜索和查询请求的延迟 | `milvus_proxy_sq_latency` |
| **Summary** | 滑动时间窗口内的分位数 | 快速获取分位数 | `-` |

**常用 Metrics 标签**

Prometheus 通过标签来区分同名指标的不同实例：

| 标签名 | 含义 | 可能值 |
|--------|------|-------|
| `node_id` | 节点唯一标识 | 由 Milvus 生成的全局唯一 ID |
| `status` | 已处理操作或请求的状态 | `abandon`, `success`, `fail` |
| `query_type` | 查询类型 | `search`, `query` |
| `msg_type` | 消息类型 | `insert`, `delete`, `search`, `query` |
| `segment_state` | 段状态 | `Sealed`, `Growing`, `Flushed`, `Flushing`, `Dropped`, `Importing` |
| `cache_state` | 缓存状态 | `hit`, `miss` |
| `cache_name` | 缓存对象名，该标签与`cache_state`一起使用 | `CollectionID`, `Schema` 等 |
| `channel_name` | 消息存储（Pulsar 或 Kafka）中的 Topic | `by-dev-rootcoord-dml_0` 等 |
| `function_name` | 处理特定请求的函数名称 | `CreateCollection`, `CreatePartition`, `CreateIndex` 等 |
| `user_name` | 用户名 | 用于认证的用户名 |
| `index_task_status` | 索引任务在元存储中的状态 | `unissued`, `in-progress`, `failed`, `finished`, `recycled` |


### 4.1.3. 可观测性组件部署实战

**检查现有环境**

首先确认 Milvus 实例是否正在运行：

```bash
docker ps | grep milvus
```

![Milvus Standalone 安装](./images/milvus-docker.png)

如果没有运行，请参考 [**1.2 Milvus 安装实战**](../ch1/ch1_2.ipynb) 中的 Docker Compose 方式启动 Milvus。

接下来我们将为这个 Milvus Standalone 实例添加完整的可观测性设施。


**1. 暴露 Milvus 的网络环境**

首先需要配置 Docker 共享网络（monitoring-network）：
```
sudo docker network create monitoring-network
```

这里需要修改之前 Milvus Docker Composer 配置文件（# 网络配置）：

```yaml
version: '3.5'

services:
  etcd:
    container_name: milvus-etcd
    image: quay.io/coreos/etcd:v3.5.18
    environment:
      - ETCD_AUTO_COMPACTION_MODE=revision
      - ETCD_AUTO_COMPACTION_RETENTION=1000
      - ETCD_QUOTA_BACKEND_BYTES=4294967296
      - ETCD_SNAPSHOT_COUNT=50000
    volumes:
      - ${DOCKER_VOLUME_DIRECTORY:-.}/volumes/etcd:/etcd
    command: etcd -advertise-client-urls=http://etcd:2379 -listen-client-urls http://0.0.0.0:2379 --data-dir /etcd
    healthcheck:
      test: ["CMD", "etcdctl", "endpoint", "health"]
      interval: 30s
      timeout: 20s
      retries: 3
    # 网络配置
    networks:
      - monitoring-network

  minio:
    container_name: milvus-minio
    image: minio/minio:RELEASE.2023-03-20T20-16-18Z
    environment:
      MINIO_ACCESS_KEY: minioadmin
      MINIO_SECRET_KEY: minioadmin
    ports:
      - "9001:9001"
      - "9000:9000"
    volumes:
      - ${DOCKER_VOLUME_DIRECTORY:-.}/volumes/minio:/minio_data
    command: minio server /minio_data --console-address ":9001"
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:9000/minio/health/live"]
      interval: 30s
      timeout: 20s
      retries: 3
    # 网络配置
    networks:
      - monitoring-network

  standalone:
    container_name: milvus-standalone
    image: milvusdb/milvus:v2.6.0-rc1
    command: ["milvus", "run", "standalone"]
    security_opt:
    - seccomp:unconfined
    environment:
      ETCD_ENDPOINTS: etcd:2379
      MINIO_ADDRESS: minio:9000
      MQ_TYPE: woodpecker
    volumes:
      - ${DOCKER_VOLUME_DIRECTORY:-.}/volumes/milvus:/var/lib/milvus
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:9091/healthz"]
      interval: 30s
      start_period: 90s
      timeout: 20s
      retries: 3
    ports:
      - "19530:19530"
      - "9091:9091"
    depends_on:
      - "etcd"
      - "minio"
    # 网络配置
    networks:
      - monitoring-network

# 网络配置
networks:
  monitoring-network:
    name: monitoring-network
    external: true
```

重启：`sudo docker compose up -d`

**2. 安装 Grafana 和 Prometheus**

首先创建一个空的目录：grafana-prometheus。

然后需要两个 yml 文件：docker-compose.yml 和 prometheus.yml

**docker-compose.yml**

这里一共有三个组件：
- prometheus
- grafana
- node-exporter

```yaml
version: '3.8'

services:
  prometheus:
    image: prom/prometheus:latest
    container_name: prometheus
    volumes:
      - ./prometheus.yml:/etc/prometheus/prometheus.yml
      - prometheus_data:/prometheus
    ports:
      - "9090:9090"
    restart: unless-stopped
    networks:
      - monitoring-network

  grafana:
    image: grafana/grafana:latest
    container_name: grafana
    ports:
      - "3000:3000"
    volumes:
      - grafana_data:/var/lib/grafana
    environment:
      - GF_SECURITY_ADMIN_USER=admin
      - GF_SECURITY_ADMIN_PASSWORD=admin
    restart: unless-stopped
    networks:
      - monitoring-network

  node-exporter:
    image: prom/node-exporter:latest
    container_name: node-exporter
    ports:
      - "9100:9100"
    volumes:
      - /proc:/host/proc:ro
      - /sys:/host/sys:ro
      - /:/rootfs:ro
    command:
      - '--path.procfs=/host/proc'
      - '--path.sysfs=/host/sys'
      - '--path.rootfs=/rootfs'
    restart: unless-stopped
    networks:
      - monitoring-network

volumes:
  prometheus_data:
  grafana_data:

networks:
  monitoring-network:
    name: monitoring-network
    external: true
```

**prometheus.yml**

Prometheus 抓取 metrics 配置（第三个即抓取 Milvus 暴露出来的数据）
- prometheus
- node-exporter
- milvus

```yaml
global:
  scrape_interval: 15s

scrape_configs:
  - job_name: 'prometheus'
    static_configs:
      - targets: ['localhost:9090']
  - job_name: 'node-exporter'
    metrics_path: /metrics
    static_configs:
      - targets: ['node-exporter:9100']
  - job_name: 'milvus'
    metrics_path: /metrics
    static_configs:
      - targets: ['milvus-standalone:9091']
```

启动：`sudo docker compose up -d`

配置完后，打开 Prometheus 网址的 targets 目录，即可看到有三个数据来源
`http://localhost:9090/targets?search=`

![prometheus-targets](./images/prometheus-targets.jpeg)

然后进入首页 `http://localhost:9090/`，就可以搜索到 milvus 相关的 Metrics

![milvus-metrics](./images/milvus-metrics.png)

**3. 在 Grafana 中可视化 Metrics**

打开 Grafna：`http://localhost:3000/connections/datasources`

首先配置 Data sources，将 Prometheus 数据导入到 Grafna 内，这里 connection

![grafana-prometheus-data](./images/grafana-prometheus-data.png)


将官网提供的 milvus-dashboard.json 下载到本地，注意 url path，里面表示 v2.6.x 的 standalone 模式的 dashboard，如果是其他模式可以相对得更换一下

```bash
curl -O https://raw.githubusercontent.com/milvus-io/web-content/refs/heads/master/v2.6.x/assets/standalone-monitoring/grafana/dashboards/milvus-standalone-dashboard.json
```

![grafna-import](./images/grafana-import.png)

![grafana-import-json](./images/grafana-import-json.png)

导入下载的 json 文件，即可看到，如果有些 metrics 数据，可以先通过 attu 导入一批测试数据，执行几次就可以看到数据了。

![grafana-milvus.png](./images/grafana-milvus.png)


**4. 安装 Loki 监控 Milvus 日志**

修改之前 grafana-prometheus 里面的 docker-compose.yml，在 services tag 下添加如下几行

```yaml
  loki:
    image: grafana/loki:latest
    container_name: loki
    ports:
      - "3100:3100"
    volumes:
      - loki_data:/loki
    restart: unless-stopped
    networks:
      - monitoring-network

  promtail:
    image: grafana/promtail:latest
    container_name: promtail
    user: "0:0"
    volumes:
    - ./promtail-config.yml:/etc/promtail/config.yml
    - /var/run/docker.sock:/var/run/docker.sock
    depends_on:
      - loki
    restart: unless-stopped
    networks:
      - monitoring-network

volumes:
  prometheus_data:
  grafana_data:
  loki_data:
```

**添加 promtail-config.yml**

```yaml
server:
  http_listen_port: 9080
  grpc_listen_port: 0

positions:
  filename: /tmp/positions.yaml

clients:
  - url: http://loki:3100/loki/api/v1/push

scrape_configs:
- job_name: system
  static_configs:
  - targets:
      - localhost
    labels:
      job: varlogs
      __path__: /var/log/*log
- job_name: milvus
  docker_sd_configs:
    - host: unix:///var/run/docker.sock
  relabel_configs:
    - source_labels: [__meta_docker_name]
      regex: milvus-standalone
      action: keep
    - source_labels: [__meta_docker_name]
      target_label: job
      replacement: milvus
    - source_labels: [__meta_docker_name]
      target_label: container
      replacement: milvus-standalone
  pipeline_stages:
    - docker: {}
```

重启组件：`sudo docker compose up -d`

然后在 Grafana Data source 内添加 Loki 数据源（同 Prometheus），Connection URL 填写：`http://loki:3100`

最后创建一个 Dashboard，数据源选择 Loki，展示面板选择 Logs 即可

![grafana-loki](./images/grafana-loki.png)

**5. 安装 Jaeger 跟踪 Milvus 使用链路**

同样进入到 grafana-prometheus 目录，修改 docker-compose.yml，在 service 下面添加上 Jaeger all-in-one 镜像

```yaml
  jaeger:
    image: jaegertracing/all-in-one:latest
    container_name: jaeger
    ports:
      - "16686:16686"  # Jaeger UI
      - "4317:4317"    # OTLP gRPC receiver
      - "4318:4318"    # OTLP HTTP receiver
      - "5778:5778"    # Jaeger agent configs
      - "14268:14268"  # Zipkin compatible endpoint
    environment:
      - COLLECTOR_OTLP_ENABLED=true
    restart: unless-stopped
    networks:
      - monitoring-network
```

添加完成后重启服务：`sudo docker compose up -d`

然后是修改 Milvus 配置，回到 Milvus 的 docker compose 所在目录。

先从目前的 Milvus-standalone 中把配置文件导出：

`docker cp milvus-standalone:/milvus/configs/milvus.yaml ./milvus.yaml`

然后修改其中的 trace 部分：

```yaml
trace:
  # trace exporter type, default is stdout,
  # optional values: ['noop','stdout', 'jaeger', 'otlp']
  exporter: jaeger
  # fraction of traceID based sampler,
  # optional values: [0, 1]
  # Fractions >= 1 will always sample. Fractions < 0 are treated as zero.
  sampleFraction: 1.0
  jaeger:
    url: http://jaeger:14268/api/traces # when exporter is jaeger should set the jaeger's URL
  otlp:
    endpoint:  # example: "127.0.0.1:4317" for grpc, "127.0.0.1:4318" for http
    method:  # otlp export method, acceptable values: ["grpc", "http"],  using "grpc" by default
    secure: true
  initTimeoutSeconds: 10 # segcore initialization timeout in seconds, preventing otlp grpc hangs forever
```

然后挂载修改后的 milvus.yaml 回去
在 docker-compose.yml 的 volumes 内添加这行，如图
`- ./milvus.yaml:/milvus/configs/milvus.yaml`

![milvus-config](./images/milvus-config.png)

然后重启：`sudo docker compose up -d`

最后打开 Jaeger（http://localhost:16686/） / Grafana（添加 Jaeger 数据源） 即可看到 Trace：

![milvus-jaeger](./images/milvus-jaeger.png)

**6. 使用配置告警规则**

因为不做很重的告警，此处就直接使用 Grafana 来配置告警规则。

Milvus 配置告警的核心就是通过 Prometheus 采集到的 Exporter 数据或者 Loki 采集到的 Logs 数据来配置表达式触发告警规则。

最简单的，我们先配置一个实例存活告警，对应的表达式如下：

`up{instance="milvus-standalone:9091"} == 0`

值默认为 1，0 代表挂了，所以我们监控 == 0 的情况

如图，在 Grafana 配置一下告警，我们可以把上面改成 ==1，就可以看到告警在 Firing 状态了：

![grafana-alter](./images/grafana-alter.png)